In [1]:
import pandas as pd
pd.set_option('display.max_colwidth',100)
import nltk

In [3]:
t_data=pd.read_csv('/content/drive/MyDrive/Ankit/EACL 2021 dec 2020/Hope speech/Tamil/tamil_hope_first_train.csv',names=["text","label","NaN"], sep='\t')
t_data.pop('NaN')
t_data

,text,label
0,Realme india product,not-Tamil
1,I also don't have tiktok hello and allnBut I'm using xiaomi,Hope_speech
2,Thalaivare..neengale inum one plus mobile vachurukinga...adha udachurunga,Non_hope_speech
3,Annee varanda thondai.. corona virus affect pannum.. neenga pathukonga. Pause koduth pasuga anne...,Hope_speech
4,5views but 18likes,Non_hope_speech
...,...,...
16155,7pm correcta erukum mg bro,Hope_speech
16156,Intha karutha mudija varaikum Neengalum ellarkittayum kondu poi sekkanum bro,Non_hope_speech
16157,And neenga adhiyavasiyam nu soldra apps like tiktok uc browser etc la oru mukkiyamana appe keday...,Non_hope_speech
16158,Daii sekram mater ku vada,Hope_speech


In [ ]:
t_data['text']=t_data['text'].str.lower()       #changing into lower case       (remove and check acc too)

In [ ]:
t_data['text']=t_data['text'].str.strip()       #remove white spaces
t_data['text']=t_data['text'].str.replace(r'\d+','')   #remove numbers
#t_data['text']=t_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
t_data['text']=t_data['text'].str.replace('[^\w\s]','')      #removing punct

In [ ]:
#removing url if any
import re
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)
t_data['text']=t_data['text'].apply(lambda x:remove_URL(x))

In [ ]:
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
labelEncode=preprocessing.LabelEncoder()
labelEncode.fit(t_data['label'])
print (labelEncode.classes_)
train_labelEncode=labelEncode.transform(t_data['label'])
label=to_categorical(np.asarray(train_labelEncode))
label

['Hope_speech' 'Non_hope_speech' 'not-Tamil']


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [ ]:
#counting labels      
 
t_data['label'].value_counts()
#must use something to balance data

Non_hope_speech    7872
Hope_speech        6327
not-Tamil          1961
Name: label, dtype: int64

In [ ]:
#words in each line
 
t_data['totalwords'] = t_data['text'].str.count(' ') + 1
t_data

,text,label,totalwords
0,realme india product,not-Tamil,3
1,i also dont have tiktok hello and allnbut im using xiaomi,Hope_speech,11
2,thalaivareneengale inum one plus mobile vachurukingaadha udachurunga,Non_hope_speech,7
3,annee varanda thondai corona virus affect pannum neenga pathukonga pause koduth pasuga anne thon...,Hope_speech,16
4,views but likes,Non_hope_speech,3
...,...,...,...
16155,pm correcta erukum mg bro,Hope_speech,5
16156,intha karutha mudija varaikum neengalum ellarkittayum kondu poi sekkanum bro,Non_hope_speech,10
16157,and neenga adhiyavasiyam nu soldra apps like tiktok uc browser etc la oru mukkiyamana appe keday...,Non_hope_speech,20
16158,daii sekram mater ku vada,Hope_speech,5


In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
tok1 = Tokenizer(char_level=False, filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n', lower = True)

In [ ]:
tok2=Tokenizer(char_level=True,filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n',lower=True)

In [ ]:
tok1.fit_on_texts(t_data['text'])
words=len(tok1.word_counts)
words

32041

In [ ]:
tok2.fit_on_texts(t_data['text'])
ltr=len(tok2.word_counts)
ltr

218

In [ ]:
Charlist=tok2.word_index
#Charlist

In [ ]:
wordlist=tok1.word_index
#wordlist

In [ ]:

encode=tok1.texts_to_sequences(t_data['text'])
print(t_data['text'][5555])
encode[5555]

இநதய எலலப பகதயல சன சல அமபபத இநதய இரணவம அனமதகக கடத


[46, 7113, 2811, 21, 697, 17525, 46, 2761, 5015, 841]

In [ ]:
##         Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=30
padd = sequence.pad_sequences(encode, maxlen=max_length, padding='post')
padd

array([[  333,    10,   213, ...,     0,     0,     0],
       [   24,   364,   355, ...,     0,     0,     0],
       [ 9657,   919,   172, ...,     0,     0,     0],
       ...,
       [   49,    30, 32038, ...,     0,     0,     0],
       [ 2947, 32041,  4225, ...,     0,     0,     0],
       [    1,  1116,   258, ...,     0,     0,     0]], dtype=int32)

In [ ]:
cencode=tok2.texts_to_sequences(t_data['text'])
cpadd=sequence.pad_sequences(cencode,maxlen=70,padding='post')

In [ ]:
# importing libraries for creating model
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding,MaxPool1D,Input
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D,LSTM,Bidirectional,GlobalAveragePooling1D
from keras.layers.merge import concatenate

#w2v Embedding

In [ ]:
embeddings_index = {}
f = open('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/Tamil/tamil w2v.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 40997 word vectors.


In [ ]:
embedding_matrix = np.zeros((words+1, 300))
for word, i in tok1.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

#Embedding layer

In [ ]:
Embedding_Layer = Embedding(words+1,300,weights=[embedding_matrix], input_length=max_length, trainable=False)
Embedding_Layer2=Embedding(words+1,300,input_length=max_length)
Embedding_Layer3=Embedding(ltr+1,300,input_length=70)

#CNN

In [ ]:
# channel 1
inputs0 = Input(shape=(70,))
embedding0 = Embedding_Layer3(inputs0)
conv0 = Bidirectional(LSTM(64))(embedding0)
drop0 = Dropout(0.5)(conv0)
#pool0 = MaxPooling1D(pool_size=2)(drop0)
#conv01=Conv1D(filters=64,kernel_size=4,activation='relu')(pool0)
#drop01=Dropout(0.5)(conv01)
#pool01=MaxPooling1D(pool_size=3,padding='same')(drop01)
#flat0= Flatten()(pool01)

In [ ]:
# channel 1
inputs1 = Input(shape=(max_length,))
embedding1 = Embedding_Layer2(inputs1)
conv1 = Bidirectional(LSTM(32))(embedding1)
drop1 = Dropout(0.5)(conv1)
#pool1 = MaxPooling1D(pool_size=2)(drop1)
#conv12=Conv1D(filters=64,kernel_size=4,activation='relu')(pool1)
#drop12=Dropout(0.5)(conv12)
#pool12=MaxPooling1D(pool_size=3,padding='same')(drop12)
#flat1 = (drop1)

In [ ]:
# channel 2
inputs2 = Input(shape=(max_length,))
embedding2 =Embedding_Layer(inputs2)
conv2 = Bidirectional(LSTM(32))(embedding2)
drop2 = Dropout(0.5)(conv2)
#pool2 = MaxPooling1D(pool_size=2)(drop2)
#flat2 =(drop2)

In [ ]:
merged = concatenate([drop0,drop1,drop2])
dense1 = Dense(64, activation='relu')(merged)
outputs = Dense(3, activation='sigmoid')(dense1)
model = Model(inputs=[inputs0,inputs1, inputs2], outputs=outputs)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 70)]         0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 70, 300)      65700       input_11[0][0]                   
____________________________________________________________________________________________

#compile

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


#Training models

In [ ]:
model.fit([cpadd,padd,padd],label,epochs=50,verbose=1,batch_size=64)

Epoch 1/50
253/253 [==============================] - 97s 348ms/step - loss: 0.5393 - acc: 0.5264
Epoch 2/50
253/253 [==============================] - 89s 351ms/step - loss: 0.4128 - acc: 0.6935
Epoch 3/50
253/253 [==============================] - 88s 349ms/step - loss: 0.2508 - acc: 0.8498
Epoch 4/50
253/253 [==============================] - 88s 350ms/step - loss: 0.1756 - acc: 0.8922
Epoch 5/50
253/253 [==============================] - 88s 350ms/step - loss: 0.1348 - acc: 0.9120
Epoch 6/50
253/253 [==============================] - 88s 347ms/step - loss: 0.1129 - acc: 0.9242
Epoch 7/50
253/253 [==============================] - 88s 348ms/step - loss: 0.0990 - acc: 0.9366
Epoch 8/50
253/253 [==============================] - 88s 346ms/step - loss: 0.0901 - acc: 0.9419
Epoch 9/50
253/253 [==============================] - 88s 349ms/step - loss: 0.0851 - acc: 0.9424
Epoch 10/50
253/253 [==============================] - 88s 347ms/step - loss: 0.0751 - acc: 0.9496
Epoch 11/50
253/253

#Development data

In [4]:
d_data=pd.read_csv('/content/drive/MyDrive/Ankit/EACL 2021 dec 2020/Hope speech/Tamil/tamil_hope_first_dev.csv',names=['text','label','nan'],sep='\t')

In [5]:
d_data=d_data[["text","label"]]
d_data

,text,label
0,Mg bro eve 6 o clock video post pannuga,Hope_speech
1,8.00 pm bro plss,not-Tamil
2,Bro ivan kitta sonna . Ivan change' pannidava poran,Hope_speech
3,@Varnajalam Mini Crafts adhum crrct thaan Akka 🤣,Non_hope_speech
4,எல்லா குழந்தைகளும் நல்ல எண்ணம்,Hope_speech
...,...,...
2013,Share karo app use panlama,Non_hope_speech
2014,@R SWAGxOP kelattu kapothi maari nee pesathada mundam kelavaa,Non_hope_speech
2015,Yow yaean yaa kelappi vidringa ipd,Non_hope_speech
2016,Hey bro. Ur doing amazing...,not-Tamil


In [6]:
d_data['label'].value_counts()

Non_hope_speech    998
Hope_speech        757
not-Tamil          263
Name: label, dtype: int64

In [ ]:
#processing as training set

d_data['text']=d_data['text'].str.lower()       #changing into lower case       (remove and check acc too)
d_data['text']=d_data['text'].str.strip()       #remove white spaces
d_data['text']=d_data['text'].str.replace(r'\d+','')   #remove numbers
#d_data['text']=d_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
d_data['text']=d_data['text'].str.replace('[^\w\s]','')      #removing punct
d_data['text']=d_data['text'].apply(lambda x:remove_URL(x))
#d_data['text'] = d_data['text'].apply(lambda x: remove_sw(x))


In [ ]:
encoded1 =tok1.texts_to_sequences(d_data['text'])
print(d_data['text'][0])
encoded1[0]

mg bro eve  o clock video post pannuga


[5, 1, 2446, 190, 262, 6, 75, 187]

In [ ]:

padded = sequence.pad_sequences(encoded1, maxlen=max_length, padding='post')
padded[0]

array([   5,    1, 2446,  190,  262,    6,   75,  187,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
cencoded1=tok2.texts_to_sequences(d_data['text'])
cpadded=sequence.pad_sequences(cencoded1,maxlen=70,padding='post')

In [ ]:

val_labelEncode=labelEncode.transform(d_data['label'])
val_label=to_categorical(np.asarray(val_labelEncode))
val_label[0]

array([1., 0., 0.], dtype=float32)

#prediction

In [ ]:
# devlopment prediction of CNN

dev_predictions = model.predict([cpadded,padded,padded])
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
dev_predictions1 = np.zeros_like(dev_predictions)
dev_predictions1[np.arange(len(dev_predictions)), dev_predictions.argmax(1)] = 1
#print(dev_predictions)
#print(dev_predictions1)

#Acuracy report

In [ ]:
#Accuracy CNN
from sklearn.metrics import classification_report
print(classification_report(val_label,dev_predictions1))

              precision    recall  f1-score   support

           0       0.49      0.52      0.50       757
           1       0.63      0.58      0.61       998
           2       0.53      0.56      0.55       263

   micro avg       0.56      0.56      0.56      2018
   macro avg       0.55      0.56      0.55      2018
weighted avg       0.56      0.56      0.56      2018
 samples avg       0.56      0.56      0.56      2018



#TESTING

In [ ]:
tx_data=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/Tamil/tamil_hope_test.csv',names=['text'],sep=',')
tx_data


,text
0,Konjam methuva pesuninganna nalarukum bro...
1,Sema bro............. தமிழன் என்று சொல்லடா தலை நிமிர்ந்து நில்லடா
2,9.52 mg bro plz crrt pannunga bro
3,Superb anna
4,Central Government kitta irunthu neraiya fund vaanga thaan Namma TN government ippadi pandrangan...
...,...
2015,Solunga bro parvala
2016,Madangowri anna 9.53s time note pannunga India vukku pirachanai chaina kutatha apparam epti nam...
2017,Enna nadu vanthu vaccine kudaka matadragalo anatha nadu mala matha nadu yela attract pannu
2018,Yes..ipdii thaan avangaluku bathil kudukanu...app banned was correct


In [ ]:
#PREprocessing
#processing as training set

tx_data['text']=tx_data['text'].str.lower()       #changing into lower case       (remove and check acc too)
tx_data['text']=tx_data['text'].str.strip()       #remove white spaces
tx_data['text']=tx_data['text'].str.replace(r'\d+','')   #remove numbers
#t_data['text']=t_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
tx_data['text']=tx_data['text'].str.replace('[^\w\s]','')      #removing punct
tx_data['text']=tx_data['text'].apply(lambda x:remove_URL(x))
#t_data['text'] = t_data['text'].apply(lambda x: remove_sw(x))


In [ ]:
encoded2 =tok1.texts_to_sequences(tx_data['text'])
padded2 = sequence.pad_sequences(encoded2, maxlen=max_length, padding='post')

In [ ]:
cencoded2=tok2.texts_to_sequences(tx_data['text'])
cpadded2=sequence.pad_sequences(cencoded2,maxlen=70,padding='post')

In [ ]:
#prediction
#prediction of CNN

predictions = model.predict([cpadded2,padded2,padded2])
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
predictions1 = np.zeros_like(predictions)
predictions1[np.arange(len(predictions)), predictions.argmax(1)] = 1
#print(dev_predictions)
print(predictions1)

[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [ ]:
pred=np.argmax(predictions1,axis=1)

In [ ]:
labelEncode.inverse_transform(pred)

array(['Hope_speech', 'Non_hope_speech', 'Hope_speech', ...,
       'Hope_speech', 'Hope_speech', 'not-Tamil'], dtype=object)

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/Tamil/tamil_hope_test.csv',names=['text'],sep=',')

In [ ]:
test['label']=labelEncode.inverse_transform(pred)
test

,text,label
0,Konjam methuva pesuninganna nalarukum bro...,Hope_speech
1,Sema bro............. தமிழன் என்று சொல்லடா தலை நிமிர்ந்து நில்லடா,Non_hope_speech
2,9.52 mg bro plz crrt pannunga bro,Hope_speech
3,Superb anna,Hope_speech
4,Central Government kitta irunthu neraiya fund vaanga thaan Namma TN government ippadi pandrangan...,Hope_speech
...,...,...
2015,Solunga bro parvala,Non_hope_speech
2016,Madangowri anna 9.53s time note pannunga India vukku pirachanai chaina kutatha apparam epti nam...,Non_hope_speech
2017,Enna nadu vanthu vaccine kudaka matadragalo anatha nadu mala matha nadu yela attract pannu,Hope_speech
2018,Yes..ipdii thaan avangaluku bathil kudukanu...app banned was correct,Hope_speech


#Into tSV

In [ ]:
test.to_csv('iiit_dwd_Tamil.tsv',index=False)